# Product Onsite Analysis

In [1]:
import pandas as pd
from prompts import *
from policy_qa import *
import pickle as pkl

/Users/jmbeck/PycharmProjects/PythonProject/MercuryInterview/src/language_models.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/jmbeck/PycharmProjects/PythonProject/MercuryInterviewMercuryInterview.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [2]:
onsite_expenses = pd.read_excel('../data/Mercury Transactions Example Safe to Share - 20250211 - Send.xlsx')

In [3]:
onsite_expenses.shape

(165, 34)

In [4]:
onsite_expenses.head()

,TRANSACTION_ID,AMOUNT,BANK_DESCRIPTION,CREATED_AT,CREATED_BY_USER_ID,KIND,NOTE,REASON_FOR_FAILURE,STATUS,TRANSACTION_PARTY_NAME,...,TRANSACTION_CATEGORY,CATEGORY_CHANGED_BY,CATEGORY_MANUALLY_UPDATED_AT,CREATED_AT.1,GL_CODE_MANUALLY_UPDATED_AT,MERCURY_CATEGORY,MERCHANT_CITY,MERCHANT_STATE,MERCHANT_COUNTRY,MERCHANT_DESCRIPTOR
0,fe1c19ec-b2a4-11ef-b1b2-41ae674c4a95,-51.95,DD *DOORDASH RUBIOSCOA,2024-12-05 01:05:17.941,a2ea8cb0-0897-11ee-a215-d363ac8f2402,CreditPaymentKind,NaN,NaN,Sent,Doordash,...,Restaurants,NaN,NaT,2024-12-05 01:06:04.939,2024-12-31 22:36:59.924,Restaurants,NaN,NaN,USA,DD *DOORDASH RUBIOSCOA
1,f3456356-d901-11ef-8852-11afbe1a64a1,-4.25,WHOLEFDS NOE 10379,2025-01-22 20:46:27.133,407ad836-1562-11ef-9830-97f5cdd6c5fb,CreditPaymentKind,NaN,NaN,Sent,Whole Foods Market,...,Grocery,NaN,NaT,2025-01-22 20:47:07.137,NaT,Grocery,SAN FRANCISCO,CA,USA,WHOLEFDS NOE 10379
2,f2573764-e363-11ef-b893-7d81186f02a4,-18.64,UBER * PENDING,2025-02-05 01:53:07.879,032070ee-0bb8-11ef-8883-1f4b394514c7,CreditPaymentKind,NaN,NaN,Sent,Uber,...,NaN,NaN,NaT,2025-02-05 01:54:09.718,NaT,NaN,SAN FRANCISCO,CA,USA,UBER * PENDING
3,f0fe4792-d830-11ef-b9a5-09adaa818648,-20.00,CLAUDE.AI SUBSCRIPTION,2025-01-21 19:50:18.493,02f1adf2-9b20-11ef-83d7-6b146210112a,CreditPaymentKind,NaN,NaN,Sent,Anthropic,...,NaN,NaN,NaT,2025-01-21 19:51:10.562,NaT,NaN,NaN,CA,USA,CLAUDE.AI SUBSCRIPTION
4,f0b422ea-e738-11ef-946e-21d6cbbc7cae,-88.93,VONS #3329,2025-02-09 22:55:21.424,a2ea8cb0-0897-11ee-a215-d363ac8f2402,CreditPaymentKind,NaN,NaN,Sent,Vons,...,NaN,NaN,NaT,2025-02-09 22:56:10.683,NaT,NaN,SANTA BARBARA,CA,USA,VONS #3329


In [5]:
def get_transaction_from_row(row):

    return {
    'id': row['TRANSACTION_ID'],
    'amount':row['AMOUNT'],
    'merchant_name':row['TRANSACTION_PARTY_NAME'],
    'merchant_description':row['MERCHANT_DESCRIPTOR'],
    'transaction_time':row['CREATED_AT'],
    'receipt_attached':row['HAS_ATTACHMENT'],
    'notes':row['NOTE']
    }



In [6]:
transactions_to_analyze = [get_transaction_from_row(row) for i,row in onsite_expenses.iterrows()]

In [7]:
#outputs = transaction_chain.batch([{'transaction':x} for x in transactions_to_analyze])
outputs = [transaction_chain.invoke({'transaction': x}) for x in transactions_to_analyze]

In [8]:
#policy_reviews = qa_chain.batch([x.json() for x in outputs])
policy_reviews = [qa_chain.invoke(x.model_dump_json()) for x in outputs]

In [9]:
policy_reviews_formatted = [output_fixing_parser.parse(x.get('result')) for x in policy_reviews]

In [10]:
# RESTART HERE
## dump files
import pickle as pkl

with open('../data/policy_reviews.pkl', 'wb') as file:
    pkl.dump(policy_reviews, file)


with open('../data/transaction_outputs.pkl', 'wb') as file2:
    pkl.dump(outputs, file2)



In [11]:
with open('../data/policy_reviews.pkl', 'rb') as file:
    policy_reviews = pkl.load(file)


with open('../data/transaction_outputs.pkl', 'rb') as file:
    outputs = pkl.load(file)

In [12]:
policy_reviews[0].get('result')

'```json\n{\n  "policy_flag": "Allowed",\n  "policy_explanation": "The transaction for food delivery from Doordash is allowable as it falls within the $100 per day meal per diem budget for individual meals, as long as it is consumed during the onsite event.",\n  "policy_sources": [\n    "$100 per day max (Inclusive of tax + tip)",\n    "Individual Meals: This includes any and all meals not provided by the office."\n  ],\n  "policy_gl_code": "Travel - Meals",\n  "recommendation": "Ensure that the transaction receipt is retained for record-keeping and reimbursement purposes."\n}\n```'

In [13]:
policy_reviews_formatted[0]

ApprovalResponse(policy_flag='Allowed', policy_explanation='The transaction for food delivery from Doordash is allowable as it falls within the $100 per day meal per diem budget for individual meals, as long as it is consumed during the onsite event.', policy_sources=['$100 per day max (Inclusive of tax + tip)', 'Individual Meals: This includes any and all meals not provided by the office.'], policy_gl_code=<GLCode.TRAVEL_MEALS: 'Travel - Meals'>, recommendation='Ensure that the transaction receipt is retained for record-keeping and reimbursement purposes.')

In [14]:
formatted_records = []
import json

for i, record in enumerate(outputs):
    record_output = json.loads(record.model_dump_json())

    record_output['policy_decision'] = policy_reviews_formatted[i].policy_flag
    record_output['policy_sources'] = policy_reviews_formatted[i].policy_sources
    record_output['policy_gl_code'] = policy_reviews_formatted[i].policy_gl_code.value
    record_output['recommendation'] = policy_reviews_formatted[i].recommendation
    formatted_records.append(record_output)

In [15]:
transactions_parsed = pd.DataFrame(formatted_records)
transactions_parsed = transactions_parsed[['transaction_id', 'transaction_category', 'alternative_categories', 'transaction_category_confidence', 'gl_code', 'alternative_gl_codes', 'gl_code_confidence', 'gl_code_explanation', 'policy_decision', 'policy_gl_code', 'policy_sources', 'recommendation']]

In [16]:
transactions_woutputs = pd.merge(onsite_expenses, transactions_parsed, how='left', left_on='TRANSACTION_ID', right_on='transaction_id')

In [17]:
transactions_woutputs.policy_decision.value_counts()

policy_decision
Allowed                      108
Disallowed                    51
More Information Required      6
Name: count, dtype: int64

In [21]:
transactions_woutputs.sort_values(by='AMOUNT')

,TRANSACTION_ID,AMOUNT,BANK_DESCRIPTION,CREATED_AT,CREATED_BY_USER_ID,KIND,NOTE,REASON_FOR_FAILURE,STATUS,TRANSACTION_PARTY_NAME,...,alternative_categories,transaction_category_confidence,gl_code,alternative_gl_codes,gl_code_confidence,gl_code_explanation,policy_decision,policy_gl_code,policy_sources,recommendation
23,d57f1bbc-e362-11ef-9295-8fe3addecbc3,-720.62,GALLERIA PARK HOTEL -,2025-02-05 01:45:09.990,a2ea8cb0-0897-11ee-a215-d363ac8f2402,CreditPaymentKind,NaN,NaN,Created,Joie De Vivre,...,"[OtherTravel, Conferences, Entertainment, Rest...",0.85,Lodging,"[Travel - Other/Wifi, Airfare, Ground Transpor...",0.90,"The expense is for lodging at a hotel, which d...",Disallowed,Lodging,[Booking outside of Navan/using your personal ...,Please provide confirmation that the lodging w...
144,1d2c2286-e357-11ef-80f9-b1386c04a22a,-715.00,SQ *WOOD THUMB,2025-02-05 00:21:16.281,02f1adf2-9b20-11ef-83d7-6b146210112a,CreditPaymentKind,Data managers onsite activity,NaN,Sent,Wood Thumb,...,"[Conferences, ProfessionalServices, Entertainm...",0.60,Other,"[Professional Fees and Services, Internal Even...",0.50,The GL code is categorized as 'Other' based on...,More Information Required,Other,[Individual Meals: This includes any and all m...,Please provide additional details about the na...
92,7523a324-e375-11ef-9dc8-2d206ff5bdc1,-700.64,TST*NOPA - DIVISADERO,2025-02-05 03:58:28.704,02f1adf2-9b20-11ef-83d7-6b146210112a,CreditPaymentKind,Data managers onsite dinner \n\nPage Leanne Lu...,NaN,Sent,Nopa,...,"[FoodDelivery, Entertainment, ProfessionalServ...",0.85,Business Meals,"[Travel - Meals, Office Meals, Customer Activa...",0.90,The expense is for a dinner with data managers...,Allowed,Travel - Meals,"[$200 per person, one time spend (in addition ...",Proceed with the reimbursement for this transa...
22,d6cb4266-e43e-11ef-83fc-852b91ab2c78,-334.70,TST* DELAROSA - DOWNTO,2025-02-06 04:00:01.448,02f1adf2-9b20-11ef-83d7-6b146210112a,CreditPaymentKind,Data managers onsite dinner Page Leanne Dan Lu...,NaN,Sent,Delarosa Marina,...,"[FoodDelivery, Entertainment, AlcoholAndBars, ...",0.90,Business Meals,"[Office Meals, Customer Activation - Meals, Te...",0.85,The GL code 'Business Meals' is appropriate as...,Allowed,Travel - Meals,"[$100 per day max (Inclusive of tax + tip), Gr...",Proceed with the reimbursement for the transac...
6,eb0e077c-b8f1-11ef-b35b-37da8d149779,-220.45,TST* THE CARRIAGE HOUS,2024-12-13 01:31:04.108,02f1adf2-9b20-11ef-83d7-6b146210112a,CreditPaymentKind,NaN,NaN,Sent,Carriage,...,"[OtherTravel, GroundTransportation, Restaurant...",0.85,Lodging,"[Travel - Other/Wifi, Airfare, Ground Transpor...",0.90,The GL code 'Lodging' is appropriate due to th...,Disallowed,Lodging,[Booking outside of Navan/using your personal ...,The transaction should be disallowed. Please r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,c9418150-e4ed-11ef-9491-2b21f9cc0fab,0.00,NaN,2025-02-07 00:52:20.634,deb5de92-b4a0-11ee-92ed-cb67e2eaa4e1,CardIntlTransactionFeeKind,NaN,The merchant canceled this transaction.,HoldReleased,Intl. Transaction Fee,...,"[Other, Utilities, ProfessionalServices, Insur...",0.85,Bank Fees,"[Other, Professional Fees and Services, Utilit...",0.80,The GL code 'Bank Fees' is appropriate as it d...,Disallowed,Bank Fees,"[If your group is smaller than this, team memb...",Consider whether this fee was necessary for th...
39,c93d0738-e4ed-11ef-ae47-61d8b6004bc6,0.00,UBER CANADA/UBERTRIP,2025-02-07 00:52:20.634,deb5de92-b4a0-11ee-92ed-cb67e2eaa4e1,CreditPaymentKind,NaN,The merchant canceled this transaction.,HoldReleased,Uber,...,"[RideshareAndTaxis, OtherTravel, VehicleExpens...",0.90,Ground Transportation,"[Airfare, Travel - Other/Wifi, Travel - Meals,...",0.85,The transaction is related to transportation s...,Allowed,Ground Transportation,[There is no need to rent a car. Please use Ub...,Proceed with the reimbursement for the Uber tr...
18,dbe32d36-cdaf-11ef-8cc1-0b51deb08768,0.28,OPENAI *CHA

In [19]:
pd.crosstab(transactions_woutputs.policy_gl_code, transactions_woutputs.policy_decision)

policy_decision,Allowed,Disallowed,More Information Required
policy_gl_code,,,
Airfare,4,0,0
Bank Fees,0,17,0
Business Meals,0,1,0
Entertainment,0,2,1
Ground Transportation,31,11,0
Lodging,0,4,0
Office Expenses,0,1,0
Other,0,3,4
Software,0,9,0


In [21]:
transactions_woutputs.to_parquet('../data/transactions_processed.parquet')